# Programmer le jeu Tetris avec un SenseHat

## Introduction
Le jeu Tetris est l'un des tout premiers jeux créés. Il s'agit de briques de différentes formes et longueurs qui descendent toute seule. Le but du jeu est de les empilés de manière méthodique pour former une ligne complète, qui disparaitra. Et ainsi de suite jusqu'à ce qu'une forme soit monté trop haut et ait bloqué l'apparition de la prochaine forme.

### Définition des variables classiques
Nous avons tout d'abord créer une variable pour chaque couleur (mais nous n'utiliserons que le rouge, le jaune, le cyan et le noir). Puis la variable sense a été assignée à SenseHat().

In [6]:
from sense_hat import SenseHat
from time import sleep, time
from random import randint, choice

## Définition des couleurs

red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
magenta = (255, 0, 255)
cyan = (0, 255, 255)
yellow = (255, 255, 0)
orange = (255, 128, 0)
white = (255, 255, 255)

ModuleNotFoundError: No module named 'sense_hat'

### Définition des variables propres à notre jeu
Voici les variables propres à un jeu comme Tetris ainsi que leur description.

In [5]:
color = (0, 0, 0) # Définition de la variable color, qui s'adaptera en fonction de la forme.
score = 0 # Le score total du joueur (le nombre de ligne complétées)
state = 1 # State = 0 quand le jeu est en statique (entre les apparitions de formes) et state = 1 quand il est en dynamique (la forme descend)
game = 1 # Game = 1 quand le jeu continue sinon c'est game over
t0 = time() # Variable de temps utile pour la descente des formes
dx = 0 # Déplacement sur l'axe X (vers le bas)
dy = 0 # Déplacement sur l'axe Y (vers le bas)
dt = 1 # Temps entre chaque descente de forme en seconde

NameError: name 'time' is not defined

### Définition des formes dans des matrices
L'écran de LED 8x8 étant assez restreint, nous avons décidés de réduire légérement les formes classiques de tetris en 3 formes principales. Ces matrices pourront être retournées pour faire une rotation de la forme par la suite. Dans les matrices, les 1 représentent les formes, donc les pixels allumés.

In [100]:
# La barre verticale de 3 en cyan
I=[[0, 1, 0],
   [0, 1, 0],
   [0, 1, 0]]

# Le L en rouge
L=[[1, 0],
   [1, 1]]

# Le carré 2x2 en jaune
O=[[1, 1],
   [1, 1]]

shapes = (I, L, O) # Liste contenant les 3 formes

P = choice(shapes) # Choisit une forme au hasard parmi les trois et la stock dans la variable P

if P == L: # Assigne la couleur de la forme choisie à la variable color
    color = red
elif P == I:
    color = cyan
else:
    color = yellow

### Définition des fonctions
#### Faire apparaître la forme
Nous arrivons dans la partie la plus compliquée, tout d'abord nous avons créé une fonction qui fait apparaître une forme au hasard en haut au milieu du SenseHat.

In [1]:
def matrix_print(M): # Affiche une forme en haut au milieu
    n = len(M)
    for y in range(n):
        for x in range(n):
            if M[y][x]==1: # Si la valeur dans la matrice est égale à 1: on allume le pixel avec la couleur correspondant à la forme.
                sense.set_pixel(3+x, y, color)

#### Faire descendre la forme

Il nous faut ensuite définir une fonction qui permet à la forme de descendre d'un pixel vers le bas. Cette fonction s'activerai donc toute les secondes automatiquement pour faire descendre la forme. On peut aussi l'activer manuellement pour descendre plus rapidement.

In [102]:
def matrix_print_down(M): # Déplace la forme vers le bas
    global dy
    global dx
    dy += 1 # Incrémente la position y de 1
    n=len(M)
    for y in range(n): # Supprime la forme actuelle
        for x in range(n):
            if 0 <= y+dy <= 7: # Vérifie si la forme peut descendre.
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy-1, black)
            elif M == [[0, 0, 0], [1, 1, 1], [0, 0, 0]] and y+dy == 8: # Permet à la barre tournée horizontalement de descendre jusqu'en bas
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy-1, black)
            else:
                dy -= 1 # Si la forme ne peut pas descendre, on diminue dy de 1 car on l'incrémente au debut comme sa la forme ne bouge pas.
    for y in range(n): # Affiche la nouvelle forme descendue de 1 si elle n'est pas en bas.
        for x in range(n):
            if 0 <= y+dy <= 7:   
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy, color)
            elif M == [[0, 0, 0], [1, 1, 1], [0, 0, 0]] and y+dy == 8:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy, color)
            else:
                state=0 # Si la forme est en bas, on passe en statique


On remarque une chose importante. La forme 'I' est un cas particulier à traiter en parallèle des deux autres formes. En effet si le I est tourné en horizontal, la dernière ligne de la matrice est vide donc on peut descendre d'une case de plus. Nous retrouverons ce cas assez souvent dans les futurs fonctions.

#### Déplacer la forme à gauche ou à droite

Mettons maintenant en place des fonctions qui permettent à la matrice de se déplacer à gauche ou à droite. Ces fonctions seront par la suite activées par le joystick.

In [ ]:
def matrix_print_left(M): # Déplace la forme vers la gauche
    global dy
    global dx
    dx -= 1
    n = len(M)
    for y in range(n): # Regarde si on est collé à une forme. Si c'est le cas, empeche d'aller plus à gauche 
        if M == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and -1 <= 3+dx <= 7:
            if M[y][1]==1 and sense.get_pixel(3+dx+1, y+dy)!=[0, 0, 0]:
                dx+=1
                return ;
        elif 0 <= 3+dx <= 7:
            if M[y][0]==1 and sense.get_pixel(3+dx, y+dy)!=[0, 0, 0]:
                dx+=1
                return ;
    for y in range(n): # Supprime la forme actuelle
        for x in range(n):
            if 0 <= 3+x+dx <= 7: # Check que la forme ne soit pas collée au bord de l'affichage
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx+1, y+dy, black)
            elif M == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and 3+x+dx == -1: # Permet à la barre verticale d'aller sur les côtés
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx+1, y+dy, black)
            else:
                dx += 1 # Si la forme est au bord, on incrémente dx car on l'a décrémenté au debut afin que la forme ne bouge pas.          
    
    for y in range(n): # Affiche la nouvelle forme décalée à gauche
        for x in range(n):
            if 0 <= 3+x+dx <= 7:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy, color)
            elif M == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and 3+x+dx == -1:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy, color)

Nous retrouvons ici à peu près la même architecture que la fonction précédente, on supprime la forme originelle et on affiche la nouvelle forme décalée. En prenant en compte le cas particulier encore une fois de la barre, qui si elle est droite peut se déplacer d'une case supplémentaire sur les côtés.
Nous avons ici un problème supplémentaire ; si la forme en rencontre une autre, elle ne peut plus avancer. Il a donc fallu déterminer si des pixels allumés gênaient le passage pour savoir si on pouvait décaler la forme.

Voici ensuite la fonction qui déplace la forme vers la droite cette fois-ci. Elle est quasiment identique à matrix_print_left à quelques détails près.

In [2]:
def matrix_print_right(M): # Déplace la forme vers la droite
    global dy
    global dx
    dx += 1
    n = len(M)
    for y in range(n): # Empêche de continuer quand elle est en colision avec une forme 
            if M == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and -1 <= 3+dx <= 6:
                if M[y][1]==1 and sense.get_pixel(3+dx+1, y+dy)!=[0, 0, 0]:
                    dx-=1
                    return ;
            if M == [[0, 0, 0], [1, 1, 1], [0, 0, 0]] and -1 <= 3+dx <= 5:
                if M[y][2]==1 and sense.get_pixel(3+dx+2, y+dy)!=[0, 0, 0]:
                    dx-=1
                    return ;
            elif 0 <= 3+dx <= 6:
                if M[y][1]==1 and sense.get_pixel(3+dx+1, y+dy)!=[0, 0, 0]:
                    dx-=1
                    return ;
    for y in range(n): # Supprime la forme actuelle
        for x in range(n):
            if 0 <= 3+x+dx <= 7:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx-1, y+dy, black)
            elif M == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and 3+x+dx == 8:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx-1, y+dy, black)
            else:
                dx -= 1 # Décrémente dx si on est collé contre un bord car on l'incrémente au début, comme sa la forme ne bouge pas.

    for y in range(n): # Affiche la forme décalée de un vers la droite
        for x in range(n):
            if 0 <= 3+x+dx <= 7:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy, color)
            elif M == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and 3+x+dx == 8:
                if M[y][x]==1:
                    sense.set_pixel(3+x+dx, y+dy, color)

#### Faire tourner la forme sur elle-même

Finalement, terminons avec la fonction qui permet de faire tourner les formes. Nous l'avons appelée rotate_90 car elle exerce une rotation de 90° de la matrice vers la droite.
Nous tenons à dire que nous avons essayé de réaliser cette fonction par nous-même pendant longtemps sans succès. Nous avons donc cherché une fonction capable de faire cela sur internet. Une partie de cette fonction (la deuxième boucle for)est donc empruntée et n'est pas de nous.

In [3]:
def rotate_90(matrix): # Tourne la matrice carrée, et donc la forme, de 90 degrés vers la droite (fonction en partie empruntée sur internet)
    n = len(matrix)
    for y in range(n): # Supprime la forme actuelle
        for x in range(n):
            if matrix[y][x]==1:
                sense.set_pixel(3+x+dx, y+dy, black)
    for layer in range((n + 1) // 2): # Définit les nouvelles coordonnées dans la matrice.
        for index in range(layer, n - 1 - layer, 1):
            matrix[layer][index], matrix[n - 1 - index][layer], \
                matrix[index][n - 1 - layer], matrix[n - 1 - layer][n - 1 - index] = \
                matrix[n - 1 - index][layer], matrix[n - 1 - layer][n - 1 - index], \
                matrix[layer][index], matrix[index][n - 1 - layer]
    for y in range(n): # Affiche la forme tournée de 90 degrés vers la droite.
        for x in range(n):
            if matrix[y][x]==1:
                sense.set_pixel(3+x+dx, y+dy, color)
    return matrix

### Les boucles du jeu (dans une boucle `while game == 1:`)
Nous entrons dans le coeur du jeu. Avant de développer ce dernier, il faut savoir que pour éviter de mettre 70 lignes de code d'un coup nous l'avons divisé en plusieurs parties. Il faut néanmoins savoir que toute les prochaines parties expliquées sont imbriquées dans une grande boucle 'while game == 1:' qui permet de répéter en boucle l'apparition des formes jusqu'au moment ou le joueur perd et la variable game passe à 0.

#### État dynamique (dans une boucle `while state == 1:`)
Juste avant d'engager cette énorme boucle game, nous avons :
`sense.clear()` et `matrix_print(P)`
Ce qui a permis de faire apparaître une forme au hasard qui marque le début du jeu.
Le jeu est donc mis par défaut en state=1 qui veut dire 'dynamique', la forme peuvent donc descendre.

##### Le joystick

In [4]:
for event in sense.stick.get_events():
             if event.action == 'pressed' and event.direction == 'middle':
                if P == [[0, 1, 0], [0, 1, 0], [0, 1, 0]]:
                    if dx == -4 or dx == 3: # Empêche de tourner la barre verticale si elle est collée contre un bord.
                        pass 
                    elif sense.get_pixel(3+dx, dy+1)!=[0, 0, 0] or sense.get_pixel(3+dx+2, dy+1)!=[0, 0, 0]: # Empèche la barre verticale de tourner lorsu'elle est collée contre un pixel allumé.
                        pass
                    else:
                        rotate_90(P)
                elif P == [[0, 0, 0], [1, 1, 1], [0, 0, 0]]:
                    if dy == 6: # Empêche de tourner la barre quand elle est horizontale et tout en bas
                        pass
                    else:
                        rotate_90(P)
                else:
                    rotate_90(P)
             elif event.direction == 'down' and event.action == 'pressed':
                matrix_print_down(P)
             elif event.direction == 'left' and event.action == 'pressed':
                matrix_print_left(P)
             elif event.direction == 'right' and event.action == 'pressed':
                matrix_print_right(P)

NameError: name 'sense' is not defined

Il s'agit ici de vérifier si le joueur appuie sur le joystick.
* Si le bouton est appuyé, il tourne la forme à l'aide de la fonction `rotate_90`, tout en traitant le cas particulier du 'I'. Si ce dernier est vertical est dans le bord, il ne peut pas devenir un I en horizontal. De même que si le I est couché et tout en bas, il ne peut pas de retourner non plus. Ces cas sont donc écarté à l'aide de `pass` qui ne fait rien.
* Le bouton du bas permet à la forme de descendre plus vite si le joueur le souhaite.
* Le bouton de gauche et de droite sont affectés à leur fonction de déplacement respective.

##### Critère pour arrêter la forme et passer en mode statique ou perdre

Il effectue ensuite un test qui permet de déterminer si la forme est tout en bas (`dy = 6 ou 5` en fonction de la forme). Si c'est le cas, on passe en mode statique (`state = 0`).

In [103]:
if P == [[0, 0, 0], [1, 1, 1], [0, 0, 0]] and dy == 6:
    state=0
elif P == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and dy == 5:
    state=0
elif P == O and dy == 6:
    state=0
elif P == L and dy == 6:
    state=0

Mais le passage en mode statique n'est pas seulement définit par le fait que la forme touche le sol. La forme doit aussi s'arrêter si elle touche une forme en dessous d'elle.

In [ ]:
n=len(P)
for x in range(n):
    if P == [[0, 1, 0], [0, 1, 0], [0, 1, 0]] and dy < 5: # Forme I et n'est pas en bas
        if sense.get_pixel(dx+4, dy+3) != [0, 0, 0]: # Si le pixel dessous est différent de noir
            if dy==0:
                game=0 # Si la forme est bloquée juste après être apparue (game over)
            state=0 # On passe en state=0 afin de faire apparaitre une nouvelle forme
    elif dy < 6 and P != [[0, 1, 0], [0, 1, 0], [0, 1, 0]]:
        if P[1][x]==1 and sense.get_pixel(x+dx+3, dy+2) != [0, 0, 0]: # Vérification sous chaque pixel de la derniere ligne de la matrice si le pixel dessous n'est pas noir.
            if dy==0: 
                game=0 # Si la forme est bloquée juste après être apparue (game over)
            state=0 # On passe en state=0 afin de faire apparaitre une nouvelle forme

#####  Le cas particulier du L

Hélàs se présente à nous un autre problème. Non pas concernant le I mais le L cette fois-ci. Le L est la seule forme concave de notre liste, il peut donc s'imbriquer dans d'autre forme.

In [ ]:
Lcomplet=0
for x in range (n): # Si les 4 pixels de l'endroit ou la matrice se situe ne sont pas noir, on passe en state=0.
    for y in range(n):
        if P == [[1, 1],[0, 1]] or P == [[1, 1],[1, 0]]:
            if sense.get_pixel(x+dx+3, y+dy)!=[0, 0, 0]:
                Lcomplet+=1
            if Lcomplet==4:
                state=0

Nous avons donc procédé de cette manière ; le L s'arrête si un pixel du dessous est autre que noir (bloc de code précédent) OU il s'arrête si les quatre cases de la matrice du L sont remplies.

##### La descente automatique des formes

Le dernier point à traiter est trivial mais l'un des plus important, c'est la descente à chaque `dt` seconde. Nous avons mis en place un test de variable plutôt qu'un `sleep(1)`. Car ce dernier aurait paralysé pendant 1 seconde toute la boucle du state 1 au lieu de continuer en boucle. Cela aurait par exemple eu comme conséquence le fait de ne pouvoir appuyer que sur un bouton par seconde sur le joystick.

In [ ]:
t = time()
if t > t0 + dt:
    matrix_print_down(P)
    t0 = t

####  État statique (dans une boucle `while state == 0:`)

##### Supprimer les lignes remplies
La première chose à vérifier lorsqu'une forme a été placée est si une ligne est remplie. Si oui, il la supprime et descend d'un pixel tout ce qu'il y a au dessus.

In [ ]:
sleep(1)
for g in range(8): # Choix de la ligne, en commencant par le bas.
    for i in range(8): # Choix de la colonne
        a = 0 # A chaque nouvelle ligne, on repasse 'a' sur 0.
        for j in range(8):
            if sense.get_pixel(7-j, 7-i) != [0, 0, 0]: # Pour chaque pixel pas noir sur la même ligne, on incrémente 'a' de 1.
                a += 1
                if a == 8: # Si a==8, cela signifie que les 8 pixels de la ligne ne sont pas noir, donc la ligne est pleine et on la supprime.
                    score += 1
                    for k in range(8): # Supression de la ligne pleine
                        sense.set_pixel(k, 7-i, black)
                    for c in reversed(range(7-i)): # Descend les lignes se situant au dessus une par une, en commencant par la plus proche.
                        for d in range(8):
                            sense.set_pixel(d, c+1, (sense.get_pixel(d, c))) # On affiche la même ligne une case plus bas 
                            sense.set_pixel(d, c, black) # On supprime la ligne

##### Faire réapparaître la prochaine forme

Il faut maintenant faire réapparaître la prochaine forme pour que le joueur puisse jouer en boucle jusqu'à ce qu'il perde.

In [ ]:
P = choice(shapes) # Choisit une nouvelle forme
         
dx = 0 # Remet les variables de déplacement à 0
dy = 0
        
if P == L: # Assigne la nouvelle couleur à color
    color = red
elif P == I:
    color = cyan
else:
    color = yellow
            
t0=time() # Remet le chrono à '0'
        
matrix_print(P) # Affiche la nouvelle matrice en haut au milieu

##### Game over ?

Avant de relancer le jeu en dynamique, il faut vérifier que game ne soit pas égal à 0, ce qui voudrait dire un game over. La seule manière que game ait été assigné à la valeur 0 se situe dans le critère pour arrêter la forme. Si la forme doit s'arrêter juste après être apparue, c'est game over.

In [ ]:
if game == 0:
    sense.show_message('Game over ! Score :', scroll_speed=0.05)
    sense.show_message(str(score), scroll_speed=0.2)
else:
    state = 1

Nous voici à la fin de la boucle `while game == 1:` si c'est game over, le programme est fini, sinon il recommence car state est passé en 1 etc ... jusqu'à un game over.

### Points à améliorer
Notre jeu est plutôt complet et marche étonnemment bien, néanmoins il resterais, si nous avions plus de temps, quelques points à améliorer.
* Le plus important est le fait que nous pouvons déplacer la forme de côté avec le joystick avant qu'elle soit apparue.
* Le second point découle du premier. Nous pouvons déplacer la forme également vers le bas avant son apparition intiale et donc faire disparaître des pixels d'autres formes qui gênaient le passage dû au processus de matrix_print_down qui supprime ce qu'il y a une case au dessus. De cette manière on peut enlever des pixels de formes qui gêne en haut au milieu du SenseHat.
* Dû au manque de temps, nous aurions pu mieux optimiser notre code et enlever pas mal de ligne à l'ensemble.